# Trustpilot Feedback Analyzer

#### Libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
from reader import feed
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import datetime as dt
import stat
import re

### Scrapper 3 - List Comprehension (better)

In [34]:
# Get Options for Company Name
company = input()

query_url = "https://www.trustpilot.com/search?query="+company

choice_request = requests.get(query_url)
choice = BeautifulSoup(choice_request.text, 'html.parser')
options = choice.select('div > h3 > a')

url_choice=[]
for o in range(len(options)):
    url_choice.append("https://www.trustpilot.com"+options[o]['href']+"?languages=de")

# Let user choose from list of options

# Use the selected Company
url_chosen = url_choice[2]  #MAKE IT FLEXIBLE LATER!
count_request = requests.get(url_chosen)
soup_init = BeautifulSoup(count_request.text, 'html.parser')

# Get number of feedback pages for website
selection=soup_init.select('#__next > div > main > div > div.styles_mainContent__nFxAv > section > div.styles_pagination__6VmQv > nav > a')

In [36]:
pages=int(selection[len(selection)-2]['aria-label'][12:])
pages

226

In [31]:
# Get Options for Company Name
company = input()

query_url = "https://www.trustpilot.com/search?query="+company

choice_request = requests.get(query_url)
choice = BeautifulSoup(choice_request.text, 'html.parser')
options = choice.select('div > h3 > a')

url_choice=[]
for o in range(len(options)):
    url_choice.append("https://www.trustpilot.com"+options[o]['href']+"?languages=de")

# Let user choose from list of options

# Use the selected Company
url_chosen = url_choice[0]  #MAKE IT FLEXIBLE LATER!
count_request = requests.get(url_chosen)
soup_init = BeautifulSoup(count_request.text, 'html.parser')

# Get number of feedback pages for website
selection=soup_init.select('#__next > div > main > div > div.styles_mainContent__nFxAv > section > div.styles_pagination__6VmQv > nav > a')
pages=int(selection[len(selection)-2]['aria-label'][12:])

# Get list of all URLs to Scrape
urllist=[]
for p in range(1,pages+1):
    if p == 1:
        urllist.append(url_chosen)
    else:
        urllist.append(url_chosen+"&page="+str(p))

# Loop through urllist to scrape all pages
revlist = []   
namelist = []
titlelist = []   
ratlist = []
datelist = []

for u in urllist:
    test = requests.get(u)
    feedbacksoup = BeautifulSoup(test.text, 'html.parser')

    # Get Names of Reviewers
    nametags= feedbacksoup.select('article > aside > div > a > div:first-child')
    namelist.extend([n.get_text() for n in nametags])

    # GET FEEDBACK CONTENT
    content=feedbacksoup.find_all('div', class_="styles_reviewContent__0Q2Tg")
    revlist.extend([p.get_text() for p in content])

    # Get Titles and clean it from the Review Content
    titles = feedbacksoup.select('div > h2') 
    titlelist.extend([len(t.get_text()) for t in titles[2:]])

    # GET REVIEW STARS
    ratings=feedbacksoup.find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
    ratlist.extend(["\n".join([img['alt'][6:7] for img in r.find_all('img', alt=True)]) for r in ratings[1:]])
        
    # Get Feedback Date
    dates = feedbacksoup.find_all('time', class_="")
    datelist.extend([d['datetime'] for d in dates])

# Create Final Dataframe with all information of the Feedbacks
review_df = pd.DataFrame({"date":datelist,"reviewer":namelist,"text":revlist, "stars":ratlist})
review_df['company'] = url_chosen[34:].split("?")[0]

# Cleaning of Text by getting rid of the title out of the text (often duplicate of the beginning of the text)
clean_content = []
for text, kill in zip(review_df['text'],titlelist):
    clean_content.append(text[kill:])

review_df['text']=clean_content
display(review_df)

IndexError: list index out of range

In [ ]:
copy=review_df.copy()

copy['date']=[pd.to_datetime(da[:10], format='%Y-%m-%d') for da in copy['date']]
copy['date'] = copy['date'].dt.date

copy.dtypes
copy.head()

### Functions

In [ ]:
#%%writefile "/Users/robertkammerer/Ironhack/Final Project/lib/functions.py"

def choose_language(language_options):
    if language_options == "English":
        return "en"
    else:
        return "de"


def get_company():
    company = input()

    language_options = ["Deutsch","English"]
    user_language=language_options[0]
    language= choose_language(user_language)

    query_url = "https://www.trustpilot.com/search?query="+company

    choice_request = requests.get(query_url)
    choice = BeautifulSoup(choice_request.text, 'html.parser')
    options = choice.select('div > h3 > a')

    url_choice=[]
    for o in range(len(options)):
        url_choice.append(f"https://www.trustpilot.com{options[o]['href']}?languages={language}")

    return url_choice


def get_basics(user_url_choice):
    # Use the selected Company
    url_chosen = user_url_choice
    count_request = requests.get(url_chosen)
    soup_init = BeautifulSoup(count_request.text, 'html.parser')

    # Get number of feedback pages for website
    pages=int(soup_init.select('#__next > div > main > div > div.styles_mainContent__nFxAv > section > div.styles_pagination__6VmQv > nav > a:nth-child(8)')[0]['aria-label'][12:])

    # Get list of all URLs to Scrape
    urllist=[]
    for p in range(1,pages+1):
        if p == 1:
            urllist.append(url_chosen)
        else:
            urllist.append(url_chosen+"&page="+str(p))

    # Get general Information for layout

    get_logo=str(soup_init.select('#__next > div > main > div > div:nth-child(3) > div.styles_wrap__bEQ8l.styles_wrapperBase__O8HY_.styles_withMobilePagination__IdPoO.styles_withHeader__ZC7W8.styles_businessUnitHeader__T06Af > div > div > div > section.styles_businessInformation__6ks_E > div.styles_summary__gEFdQ > div.profile-image_imageWrapper__kDdWe > a > picture > img'))
    logo_link=re.search("(?P<url>https?://[^\s]+)", get_logo).group("url")[:-4]

    overall_rating=soup_init.select('#business-unit-title > div > div > p')[0].get_text()

    features =[elem.get_text() for elem in soup_init.find_all('div', class_='styles_activityCard__aqGtn')]
    claimed = features[0]
    requestor = features[1]
    resrate= features[3]
    restime=features[4]

    return urllist, logo_link, overall_rating, claimed, requestor, resrate, restime


def create_dataframe(urllist):

    revlist = []   
    namelist = []
    titlelist = []   
    ratlist = []
    datelist = []

    for u in urllist:
        test = requests.get(u)
        feedbacksoup = BeautifulSoup(test.text, 'html.parser')

        # Get Names of Reviewers
        nametags= feedbacksoup.select('article > aside > div > a > div:first-child')
        namelist.extend([n.get_text() for n in nametags])

        # GET FEEDBACK CONTENT
        content=feedbacksoup.find_all('div', class_="styles_reviewContent__0Q2Tg")
        revlist.extend([p.get_text() for p in content])

        # Get Titles and clean it from the Review Content
        titles = feedbacksoup.select('div > h2') 
        titlelist.extend([len(t.get_text()) for t in titles[2:]])

        # GET REVIEW STARS
        ratings=feedbacksoup.find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
        ratlist.extend(["\n".join([img['alt'][6:7] for img in r.find_all('img', alt=True)]) for r in ratings[1:]])
            
        # Get Feedback Date
        dates = feedbacksoup.find_all('time', class_="")
        datelist.extend([d['datetime'] for d in dates])

    # Create Final Dataframe with all information of the Feedbacks
    review_df = pd.DataFrame({"date":datelist,"reviewer":namelist,"text":revlist, "stars":ratlist})
    review_df['company'] = url_chosen[33:] 

    # Cleaning of Text by getting rid of the title out of the text (often duplicate of the beginning of the text)
    clean_content = []
    for text, kill in zip(review_df['text'],titlelist):
        clean_content.append(text[kill:])

    review_df['text']=clean_content

    return review_df


def file_path(url_chosen):
    filepath_input=url_chosen[30:].split("?")[0][4:-3].replace("www.","")
    filepath=f"/Users/robertkammerer/Ironhack/Final Project/data/{filepath_input}.csv"
    return filepath


def decision_path(url_choice,filepath):
    if os.path.exists(filepath):
        if dt.date.fromtimestamp(os.path.getmtime(filepath)) > dt.date.today()- dt.timedelta(days=7):
            # Database is up to date -> use existing file
            df=pd.read(filepath)
        else:
            # Database is outdated -> delete old file, scrape new data and save it
            os.remove(filepath)
            print('old data removed')
            scrapelist=get_basics(url_choice)
            df=create_dataframe(scrapelist)
            df.to_csv(f"/Users/robertkammerer/Ironhack/Final Project/data/{filepath_input}.csv", index=False)
    else:
        scrapelist=get_basics(url_choice)
        df=create_dataframe(scrapelist)
        df.to_csv(f"/Users/robertkammerer/Ironhack/Final Project/data/{filepath_input}.csv", index=False)

    return df

In [ ]:
review_df.head()

#### Data Prep

In [11]:
# Convert Dates
review_df['date']=[pd.to_datetime(da[:10], format='%Y-%m-%d') for da in review_df['date']]
#review_df['date'] = review_df['date'].dt.date

# Add Cluster for positive/negative feedback
review_df["pos/neg"]= review_df['stars'].astype(str).apply(lambda x: "positive" if x =="5" else ("negative" if x in ["1","2"] else "neutral"))

# Convert stars to int
review_df['stars'] = review_df['stars'].astype(int)

# Add year and month
review_df['year'] = [elem.year for elem in review_df['date']]
review_df['year/month'] = review_df['date'].dt.strftime('%Y-%m')
review_df['month'] = [elem.month for elem in review_df['date']]

In [ ]:
# Percentages for Ratings

five_stars=len(review_df[review_df['stars']==5])/len(review_df['stars'])
four_stars=len(review_df[review_df['stars']==4])/len(review_df['stars'])
three_stars=len(review_df[review_df['stars']==3])/len(review_df['stars'])
two_stars=len(review_df[review_df['stars']==2])/len(review_df['stars'])
one_stars=len(review_df[review_df['stars']==1])/len(review_df['stars'])

print(five_stars)
print(four_stars)
print(three_stars)
print(two_stars)
print(one_stars)

csat = len(review_df[review_df['stars']>=4])/len(review_df['stars'])
csat


In [ ]:
display(review_df)
print(review_df.dtypes)

#### Visualisations

In [ ]:
### Grouping
display(review_df.head())
review_df[['year','stars']].groupby('year').mean()
review_df[['year/month','pos/neg','stars']].groupby(['year/month']).agg({'pos/neg':'size', 'stars':'mean'}).rename(columns={'pos/neg':'count', "stars":'mean'})

In [ ]:
plt.figure(figsize=(16,9), dpi= 80)
sns.histplot(x=review_df['date'], hue=review_df['stars'], hue_order=[1,2,3,4,5])
plt.show()

In [ ]:
# Prepare data
x_var = 'displ'
groupby_var = 'class'
df_agg = df.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [df[x_var].values.tolist() for i, df in df_agg]

# Draw
plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.Spectral(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, 30, stacked=True, density=False, color=colors[:len(vals)])

# Decoration
plt.legend({group:col for group, col in zip(np.unique(df[groupby_var]).tolist(), colors[:len(vals)])})
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.xlabel(x_var)
plt.ylabel("Frequency")
plt.ylim(0, 25)
plt.xticks(ticks=bins[::3], labels=[round(b,1) for b in bins[::3]])
plt.show()

In [ ]:
start = None
end = None
axis_unit=['month', 'year']
unit = axis_unit[1]

df = review_df.copy()
df

In [ ]:
def histogram(start, end, unit, df):

    import seaborn as sns
    import matplotlib.pyplot as plt
    import streamlit as st
    import time
    import datetime as dt
    import pandas as pd
    import numpy as np


    #Check the date input
    if start == None:
        start=pd.Timestamp(dt.datetime.today()- dt.timedelta(days=5000))
    else:
        start=pd.Timestamp(start)

    if end == None:
        end=pd.Timestamp(dt.datetime.today())
    else:
        end=pd.Timestamp(end)

    db_filtered = df[(df['date']>=start) & (df['date']<=end)]

    if unit == "month":
        db_final =db_filtered[['year/month', 'stars']]
    else:
        db_final = db_filtered[['year','stars']]


    fig1, ax1 = plt.subplots(figsize=(20,10))
    sns.histplot(x=db_final.iloc[0:,0], hue=db_final['stars'])
    ax1.set_title("Number of Reviews over time")

    st.pyplot(fig1)

In [ ]:
def histogram(start, end, unit, df):

    import seaborn as sns
    import matplotlib.pyplot as plt
    import streamlit as st
    import time
    import datetime as dt
    import pandas as pd
    import numpy as np


    #Check the date input
    if start == None:
        start=pd.Timestamp(dt.datetime.today()- dt.timedelta(days=5000))
    else:
        start=pd.Timestamp(start)

    if end == None:
        end=pd.Timestamp(dt.datetime.today())
    else:
        end=pd.Timestamp(end)

    db_filtered = df[(df['date']>=start) & (df['date']<=end)]

    if unit == "month":
        db_final =db_filtered[['year/month', 'stars', 'pos/neg']].groupby(['year/month']).agg({'pos/neg':'size', 'stars':'mean'}).rename(columns={'pos/neg':'count', "stars":'mean'})
    else:
        db_final = db_filtered[['year','stars', 'pos/neg']].groupby(['year']).agg({'pos/neg':'size', 'stars':'mean'}).rename(columns={'pos/neg':'count', "stars":'mean'})

    fig2, ax1 = plt.subplots(figsize=(15,5))
    sns.lineplot(x=db_final.iloc[0:,0], y= db_final['mean'])
    ax1.set_title("Avg. review score cohorts")

    st.pyplot(fig2)    



#### Testting


In [179]:
start=pd.Timestamp(dt.datetime.today()- dt.timedelta(days=5000))
end=pd.Timestamp(dt.datetime.today())

In [182]:
db_final = copy[['year','stars', 'pos/neg']].groupby(['year']).agg({'pos/neg':'size', 'stars':'mean'}).rename(columns={'pos/neg':'count', "stars":'mean'})

In [204]:
check = db_final.reset_index()
check.dtypes
check

,year,count,mean
0,2020,45,4.911111
1,2021,96,4.614583
2,2022,44,4.545455


In [227]:
copy
copy['date']=copy['date'].astype(str)
copy2= copy.groupby(['year','stars']).agg({'year/month':'size'}).reset_index()
copy2

,year,stars,year/month
0,2020,4,4
1,2020,5,41
2,2021,1,5
3,2021,2,3
4,2021,3,2
5,2021,4,4
6,2021,5,82
7,2022,1,2
8,2022,2,1
9,2022,3,3


# WordCLoud Basis

In [4]:
## Get data

review_df=pd.read_csv('/Users/robertkammerer/Ironhack/Final Project/data/mcmakler-finance.csv')
# Convert Dates
review_df['date']=[pd.to_datetime(da[:10], format='%Y-%m-%d') for da in review_df['date']]
#review_df['date'] = review_df['date'].dt.date

# Add Cluster for positive/negative feedback
review_df["pos/neg"]= review_df['stars'].astype(str).apply(lambda x: "positive" if x =="5" else ("negative" if x in ["1","2"] else "neutral"))

# Convert stars to int
review_df['stars'] = review_df['stars'].astype(int)

# Add year and month
review_df['year'] = [elem.year for elem in review_df['date']]
review_df['year/month'] = review_df['date'].dt.strftime('%Y-%m')
review_df['month'] = [elem.month for elem in review_df['date']]
copy=review_df.copy()
copy

,date,reviewer,text,stars,company,pos/neg,year,year/month,month
0,2022-04-10,A. Kahriman,Ich bin so froh dass wir einen so tollen und k...,5,www.mcmakler-finance.de,positive,2022,2022-04,4
1,2022-04-09,Kunde,Totale Zufriedenheit mit der angebotenen Diens...,5,www.mcmakler-finance.de,positive,2022,2022-04,4
2,2022-04-08,charly d,Termine werden nicht wahrgenommen und es erfol...,1,www.mcmakler-finance.de,negative,2022,2022-04,4
3,2022-04-05,Daniel,Ich kann Herrn Billor zu 110% nur weiterempfeh...,5,www.mcmakler-finance.de,positive,2022,2022-04,4
4,2022-04-05,Kunde,"Alles top, sehr gute Unterstützung und schnell...",5,www.mcmakler-finance.de,positive,2022,2022-04,4
...,...,...,...,...,...,...,...,...,...
180,2020-01-28,Chris T,Sehr freundliche und kompetente Beratung. Die ...,5,www.mcmakler-finance.de,positive,2020,2020-01,1
181,2020-01-27,FalcoW,-schnelle Bearbeitung meines Anliegens-gute Ko...,5,www.mcmakler-finance.de,positive,2020,2020-01,1
182,2020-01-21,Nata Hofmann,"konnten am Ende leider das Haus nicht kaufen, ...",5,www.mcmakler-finance.de,positive,2020,2020-01,1
183,2020-01-14,Georg Bauer,"Finanzierung hat super geklappt. Top Beratung,...",5,www.mcmakler-finance.de,positive,2020,2020-01,1


In [5]:
copy = copy.dropna()
positive=copy[copy['pos/neg']=="positive"]
negative=copy[copy['pos/neg']=="negative"]

In [46]:
def clean_up(s):
    s = str(s).lower().replace("\\","").replace("_"," ")
    # Replace urls by ''
    s = re.sub(r'http\S+', ' ' , s) 
    s = re.sub(r'\W+',' ',s) # Replace everything non-alpahnumeric by ' '
    s = re.sub(r'\d+',' ',s) # Replace one or more digits by  ' '
    s = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", s) # Replace e-mails by ''
    # Replace html tags by ''
    s = re.sub(r'\s+',' ',s) # Replace one or more whitespaces by  ' '

    return s

#### NLTK


In [294]:
import nltk
from bs4 import BeautifulSoup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robertkammerer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robertkammerer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/robertkammerer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/robertkammerer/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/robertkammerer/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [2]:
def clean_up(s):
    s = str(s).lower().replace("\\","").replace("_"," ")
    # Replace urls by ''
    s = re.sub(r'http\S+', ' ' , s) 
    s = re.sub(r'\W+',' ',s) # Replace everything non-alpahnumeric by ' '
    s = re.sub(r'\d+',' ',s) # Replace one or more digits by  ' '
    s = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", s) # Replace e-mails by ''
    # Replace html tags by ''
    s = re.sub(r'\s+',' ',s) # Replace one or more whitespaces by  ' '
    return s

def tokenize(s):
    from nltk.tokenize import word_tokenize
    tokenized = word_tokenize(s, language='german')
    return tokenized

def remove_stopwords(l):
    from nltk.corpus import stopwords
    stop_words = list(stopwords.words('german'))
    for i in range(len(stop_words)):
      stop_words[i] = re.sub(r"\s*'\s*\w*","",stop_words[i])

    cleaned = [(word) for word in l if word not in stop_words]

    return cleaned


def nltk_cleaner(column):

    #Clean Data
    cleaned_column=clean_up(column)

    #Run Tokenizer
    toks=tokenize(cleaned_column)

    #Run Hanover Tagger
    from HanTa import HanoverTagger as ht
    tagger = ht.HanoverTagger('morphmodel_ger.pgz')
    lemmed = [tagger.tag_sent(toks,taglevel= 1)]
    lemmed
    hanta_lemma=[word[1] for sent in lemmed for word in sent]

    #Stopwords and corpus
    hanta_lemma_stop = remove_stopwords(hanta_lemma)
    

    return hanta_lemma_stop

# Create Corpus
hanta_text = " ".join(word for word in hanta_lemma_stop)
hanta_text

In [189]:
positive['nltk_processed']=positive['text'].apply(lambda x: nltk_cleaner(x))

/var/folders/gg/ydzntksx7jl3t92y3_bqsw140000gn/T/ipykernel_7619/25145169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive['nltk_processed']=positive['text'].apply(lambda x: nltk_cleaner(x))


In [9]:
hanta_lemma_stop=positive['text'].apply(lambda x: nltk_cleaner(x))
hanta_text = " ".join(word for word in hanta_lemma_stop)
hanta_text

'froh toll kompetent Berater wirklich lange begleiten wichtig Angelegenheit immer erreichbar immer Mcmakler Finance entscheiden Herr hinkelmann schon Freund bekannt empfehlen vielen Dank total Zufriedenheit angeboten Dienstleistung Erstbetatung Abschluss Finanzierung einzeln Punkt aufklären ganz Weg begleiten genau stellen Tätigkeit Finanzberater prompt kompetent zuverlässig Herr Billor weiterempfehlen Anfang stehen s hilfsbereit motivieren rundum hoch Servicequalität erster Anruf Notartermin fühlen immer gut Hand aufheben top gut Unterstützung schnell feedback groß Lob gelten Herr Frank Pfister unendlich Geduld immer offen Ohr tröstend beruhigend Wort kennen Herr pfister seit Mitte Januar nämlich passieren nachdem Haus finden gelten nämlich Tempo maklerin groß Druck ausüben Finanzierung Herr pfister innerhalb weniger Tag durchbringen positiv Entscheidung seitens Bank Verkäufer treten Verkauf zurück inklusive Herr pfister Schlag Magengrube stehen Rat tun Seite müssen nochmal vorne begi

#### Spacy

In [10]:
def spacy_cleaner(column):
    #Clean Data
    cleaned_column=column.apply(lambda x: clean_up(x))

    #Create clean corpus
    clean_corpus = ' '.join(str(v) for v in cleaned_column)
    clean_corpus = re.sub(r'\s+',' ', clean_corpus)

    #Use Spacy pretrained model
    import spacy
    from spacy.lang.de.stop_words import STOP_WORDS

    spacy_nlp= spacy.load('de_core_news_md')
    spacy_stopwords = STOP_WORDS
    spacy_stopwords = list(spacy_stopwords)

    doc = spacy_nlp(clean_corpus)

    # manual stopwords
    spacy_stopwords = STOP_WORDS
    spacy_stopwords = list(spacy_stopwords)
    spacy_stopwords

    lemmas = []
    for token in doc:
        if token.orth_ not in spacy_stopwords:
            lemmas.append(token.lemma_)

    # Corpus Creation
    lemmas_text = ' '.join(str(v) for v in lemmas)

    return lemmas_text


In [11]:
spacy_cleaner(positive['text'])

'froh toll kompetent berater begleiten wichtig angelegenheiten erreichbar mcmakler finance entscheiden herrn hinkelmann freunden bekannt empfehlen totale zufriedenheit angeboten dienstleistung erstbetatung abschluss finanzierung einzeln punkt aufklären weg begleiten genau stellen tätigkeit finanzberaters prompt kompetent zuverlässig herrn billor weiterempfehlen anfang stehen s hilfsbereit motivieren rundum hoch servicequalität anruf notartermin fühlen gut händen aufheben top unterstützung schnell feedback lob gelten herrn frank pfister unendliche geduld offen ohr tröstend beruhigend worte kennen herrn pfister mitte januar nämlich passieren haus finden gelten nämlich tempo maklerin druck ausüben finanzierung herr pfister innerhalb durchbringen positiv entscheidung seitens bank verkäufer treten verkauf inklusive herrn pfister schlag magengrube stehen rat seite nochmal vorne beginnen haussuche neu finanzierungsanfrage schwierig grund hoch nachfragen krieg ukraine herr pfister ruhig geduld

In [13]:


def my_wordcloud(data, title=None):
    from wordcloud import WordCloud, STOPWORDS
    stopwords = STOPWORDS   

    wordcloud=WordCloud(
        background_color='white',
        stopwords = stopwords,
        max_words=20,
        max_font_size=40,
        scale=5,
        random_state=2).generate(str(data))

    fig = plt.figure(1, figsize=(20,20))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

my_wordcloud(lemmas_text)


NameError: name 'lemmas_text' is not defined

In [81]:
for word in doc:
  print(token.text, token.pos_)

ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON
ihnen PRON

In [16]:
spacy_stopwords = STOP_WORDS
spacy_stopwords = list(spacy_stopwords)
spacy_stopwords

['hatte',
 'also',
 'mancher',
 'rechtes',
 'satt',
 'wem',
 'neue',
 'wen',
 'des',
 'ganze',
 'indem',
 'seines',
 'tage',
 'etwas',
 'viel',
 'anders',
 'wollten',
 'seien',
 'ihre',
 'welches',
 'dieser',
 'grosse',
 'deinem',
 'jener',
 'sein',
 'alle',
 'ach',
 'zum',
 'dieselben',
 'schlecht',
 'zweiter',
 'zehntes',
 'recht',
 'kleines',
 'zehn',
 'wohl',
 'wegen',
 'wollen',
 'dementsprechend',
 'etwa',
 'dürfen',
 'zusammen',
 'weniger',
 'endlich',
 'gehabt',
 'auf',
 'jedermanns',
 'vielen',
 'vierte',
 'dafür',
 'wo',
 'wart',
 'einander',
 'keinem',
 'ihm',
 'nichts',
 'zeit',
 'im',
 'doch',
 'meine',
 'wenig',
 'ihn',
 'sollten',
 'um',
 'statt',
 'macht',
 'welchen',
 'ausserdem',
 'jemanden',
 'tel',
 'ehrlich',
 'darf',
 'warum',
 'beiden',
 'haben',
 'einige',
 'hast',
 'dieselbe',
 'keiner',
 'mit',
 'einiger',
 'können',
 'nicht',
 'neunte',
 'achte',
 'darin',
 'allerdings',
 'diejenige',
 'ob',
 'weit',
 'dich',
 'zwar',
 'vierten',
 'unser',
 'dies',
 'soll',
 

In [15]:
from spacy.lang.de.stop_words import STOP_WORDS

# Stopwords

In [50]:
rk_stopwords.append('frau')

In [39]:
# Safe List of Stopwords
with open('/Users/robertkammerer/Ironhack/Final Project/lib/stopwords.csv', "w") as file:
    file_lines = "\n".join(rk_stopwords)
    file.write(file_lines)

In [30]:
#Open list of stopwords
file = open("/Users/robertkammerer/Ironhack/Final Project/lib/stopwords.csv", "r")
file_lines = file.read()
rk_stopwords = file_lines.split("\n")

In [51]:
rk_stopwords

['hatte',
 'also',
 'mancher',
 'rechtes',
 'satt',
 'wem',
 'neue',
 'wen',
 'des',
 'ganze',
 'indem',
 'seines',
 'tage',
 'etwas',
 'viel',
 'anders',
 'wollten',
 'seien',
 'ihre',
 'welches',
 'dieser',
 'grosse',
 'deinem',
 'jener',
 'sein',
 'alle',
 'ach',
 'zum',
 'dieselben',
 'schlecht',
 'zweiter',
 'zehntes',
 'recht',
 'kleines',
 'zehn',
 'wohl',
 'wegen',
 'wollen',
 'dementsprechend',
 'etwa',
 'dürfen',
 'zusammen',
 'weniger',
 'endlich',
 'gehabt',
 'auf',
 'jedermanns',
 'vielen',
 'vierte',
 'dafür',
 'wo',
 'wart',
 'einander',
 'keinem',
 'ihm',
 'nichts',
 'zeit',
 'im',
 'doch',
 'meine',
 'wenig',
 'ihn',
 'sollten',
 'um',
 'statt',
 'macht',
 'welchen',
 'ausserdem',
 'jemanden',
 'tel',
 'ehrlich',
 'darf',
 'warum',
 'beiden',
 'haben',
 'einige',
 'hast',
 'dieselbe',
 'keiner',
 'mit',
 'einiger',
 'können',
 'nicht',
 'neunte',
 'achte',
 'darin',
 'allerdings',
 'diejenige',
 'ob',
 'weit',
 'dich',
 'zwar',
 'vierten',
 'unser',
 'dies',
 'soll',
 